In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

In [2]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv('OPEN_AI_KEY')
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

First, initialize your LLM

In [3]:
# TODO: initialize your LLM
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=2000)

c:\Users\freda\miniconda3\Lib\site-packages\langchain_community\llms\openai.py:254: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\freda\miniconda3\Lib\site-packages\langchain_community\llms\openai.py:1073: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


Then, load reviews from tv-reviews.csv

In [4]:
# TODO: load your documents
docs = CSVLoader(file_path='./tv-reviews.csv').load()
print(docs)


[Document(metadata={'source': './tv-reviews.csv', 'row': 0}, page_content="TV Name: Imagix Pro\nReview Title: Amazing Picture Quality\nReview Rating: 9\nReview Text: I recently purchased the Imagix Pro and I am blown away by its picture quality. The colors are vibrant and the images are crystal clear. It feels like I'm watching movies in a theater! The sound is also impressive, creating a truly immersive experience. Highly recommended!"), Document(metadata={'source': './tv-reviews.csv', 'row': 1}, page_content="TV Name: Imagix Pro\nReview Title: Impressive Features\nReview Rating: 8\nReview Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has convenient shortcuts. The slim design is sleek and fits perfectly in my living room. The only downside is that the sound could be better, but overall, I'm satisfied."), Document(metadat

Split the documents you loaded into smaller chunks

In [5]:
# TODO: use a Text Splitter to split the documents into chunks
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

splitter_docs = splitter.split_documents(docs)

Now, initialize your embeddings

In [6]:
# TODO: initialize your embeddings model
embeddings = OpenAIEmbeddings()

C:\Users\freda\AppData\Local\Temp\ipykernel_10376\872630394.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


Initialize your vector db with your embeddings model and populate with your text chunks

In [7]:
# TODO: populate your vector database with the chunks
db = Chroma.from_documents(splitter_docs, embeddings)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

Query your vector database for 5 most semantically similar chunks

In [ ]:
query = """
    Based on the reviews in the context, tell me what people liked about the picture quality.
    Make sure you do not paraphrase the reviews, and only use the information provided in the reviews.
    """
# find top 5 semantically similar documents to the query
use_chain_helper = False

if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt=prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

Combined, they should provide enough information to answer our question about picture quality

In [ ]:
# query your LLM with the query and the top 5 documents